In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import seaborn as sns
from collections import Counter
from dateutil.relativedelta import relativedelta
import datetime
import copy

In [ ]:
pd.set_option('max_columns', None)

In [3]:
start = time.time()
df_train = pd.read_csv("raw_data/train_data.csv", parse_dates=[1])
df_test = pd.read_csv("raw_data/test_set_VU_DM.csv", parse_dates=[1])
end = time.time()
print("Train set loaded in ", end - start, " seconds.")

Train set loaded in  39.5938286781311  seconds.


In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_train.head(3)

In [ ]:
df_test.shape

In [8]:
def click_book_rates(data):
    #Create dataframe of click and booking rates per property
    clone_data = copy.deepcopy(data)
    
    counts = pd.DataFrame(clone_data['prop_id'].value_counts(sort=False).reset_index())
    counts.columns = ['prop_id', 'counts']

    clicks = pd.DataFrame(clone_data.groupby('prop_id')['click_bool'].sum().reset_index())
    clicks.columns = ['prop_id', 'clicks']

    bookings = pd.DataFrame(clone_data.groupby('prop_id')['booking_bool'].sum().reset_index())
    bookings.columns = ['prop_id', 'bookings']

    temp = pd.merge(counts, clicks, left_on='prop_id', right_on='prop_id')
    df_rates = pd.merge(temp, bookings, left_on='prop_id', right_on='prop_id')

    df_rates['click_rate'] = df_rates['clicks'] / df_rates['counts']
    df_rates['booking_rate'] = df_rates['bookings'] / df_rates['counts']
    df_rates.drop(columns=['counts', 'clicks', 'bookings'], axis=1, inplace=True)

    return df_rates

In [9]:
df_rates = click_book_rates(df_train)

In [12]:
df_rates

,prop_id,click_rate,booking_rate
0,4098,0.015625,0.015625
1,8196,0.000000,0.000000
2,12294,0.052632,0.052632
3,16392,0.000000,0.000000
4,20490,0.000000,0.000000
...,...,...,...
129108,24564,0.062500,0.000000
129109,16376,0.080000,0.060000
129110,12282,0.100000,0.100000
129111,8188,0.000000,0.000000


In [13]:
df_test = pd.merge(df_test,df_rates,on='prop_id',how='left')

In [14]:
df_test.isnull().sum()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating        4705752
visitor_hist_adr_usd           4704559
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7266
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1088032
prop_log_historical_price            0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
srch_query_affinity_score      4641025
orig_destination_distance      1608679
random_bool              

In [11]:
df_rates.to_csv('df_rates.csv')

## Date Feature Engineering

In [ ]:
df_train['date_time']

In [4]:
for dates in df_train['date_time']:
    print(datetime.datetime.date(dates))
    print(datetime.datetime.date(dates).weekday())
    break;

2013-04-04
3


In [5]:
df_train['day_of_the_week'] = df_train['date_time'].apply((lambda x: datetime.datetime.date(x).weekday()))
df_train['day'] = df_train['date_time'].apply((lambda x: datetime.datetime.date(x).day))
df_train['month'] = df_train['date_time'].apply((lambda x: datetime.datetime.date(x).month))
df_train['year'] = df_train['date_time'].apply((lambda x: datetime.datetime.date(x).year))
df_train['week'] = df_train['date_time'].apply((lambda x: datetime.datetime.date(x).isocalendar()[1]))

In [6]:
df_train.head(-1)

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,day_of_the_week,day,month,year,week
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,0.0,NaN,0,NaN,0,3,4,4,2013,14
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,0.0,NaN,0,NaN,0,3,4,4,2013,14
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,0.0,NaN,0,NaN,0,3,4,4,2013,14
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,0.0,5.0,0,NaN,0,3,4,4,2013,14
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,0.0,NaN,0,NaN,0,3,4,4,2013,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958341,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,55110,3,1.0,...,NaN,NaN,0,NaN,0,6,30,6,2013,26
4958342,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,77700,3,4.0,...,NaN,NaN,0,NaN,0,6,30,6,2013,26
4958343,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,88083,3,4.0,...,NaN,NaN,0,NaN,0,6,30,6,2013,26
4958344,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,94508,3,3.5,...,NaN,NaN,0,NaN,0,6,30,6,2013,26


## Dataset Statistics

In [ ]:
df_train.groupby('prop_id')['prop_location_score1'].mean()

In [ ]:
df_train[df_train['prop_id']==1]['prop_log_historical_price']

In [ ]:
df_train.groupby('srch_id')['prop_location_score1'].mean()

In [ ]:
df_train.shape

In [ ]:
#Number of properties
len(set(df_train.prop_id))

In [ ]:
#Number of searches
len(set(df_train.srch_id))

In [ ]:
#Number of search Destinations
len(set(df_train.srch_destination_id))

In [ ]:
#Number of search Destinations
len(set(df_train.site_id))

In [ ]:
#visitor_location_country_id
len(set(df_train.prop_country_id))

In [ ]:
#visitor_location_country_id
len(set(df_train.visitor_location_country_id))

In [ ]:
(df_train.price_usd > 10000).value_counts()

In [ ]:
(df_train.price_usd == 0).value_counts()

In [ ]:
df_train.price_usd.plot(kind='hist', ylim=(0,25))

In [ ]:
df_train.date_time.describe()

In [ ]:
pos_clicked = df_train['position'].where(df_train['click_bool']==1)
pos_booked = df_train['position'].where(df_train['booking_bool']==1)

In [ ]:
pos_clicked

In [ ]:
pos_clicked = pos_clicked.value_counts(normalize=True)
pos_clicked.index = pos_clicked.index.map(int)
pos_booked = pos_booked.value_counts(normalize=True)
pos_booked.index = pos_booked.index.map(int)

fig, ax = plt.subplots(figsize=[15,5])
#x = np.arange(1, 41)
x = np.arange(1, 41)
ax.bar(x-0.2, pos_clicked[x], width=0.4, label='Clicked')
ax.bar(x+0.2, pos_booked[x], width=0.4, label='Booked')
ax.set_xticks(x)
ax.set_yticklabels(['{:.1%}'.format(i) for i in np.linspace(0, 0.2, 9)])

plt.legend()
plt.show()

In [ ]:
x = np.arange(1, 41)
fig, ax1 = plt.subplots(figsize=(10, 10))
#sns.set_theme(style="whitegrid")

sns.barplot(x=x, y=pos_booked, ax=ax1, label='Booked')
sns.barplot(x=x, y=pos_clicked, ax=ax1,edgecolor=".2", label='Clicked')
#ax.bar(x-0.2, pos_clicked[x], width=0.4, label='Clicked')
#ax.bar(x+0.2, pos_booked[x], width=0.4, label='Booked')
#ax1.set_xticks(x)
ax1.set_yticklabels(['{:.1%}'.format(i) for i in np.linspace(0, 0.2, 9)])
plt.legend()
plt.show()




months = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

def name(x):
    if x[2] == 15:
        return '%s %d' % (months[x[1]], x[0])
    else:
        return ''

df2 = df_train

dates = [(a.year, a.month, a.day) for a in df2.date_time]
c = sorted(Counter(dates).items(), key=lambda kv: kv[0][0]*10000 + kv[0][1]*100 + kv[0][2])
datesort = [kv[0][0]*10000 + kv[0][1]*100 + kv[0][2] for kv in c]
names = [name(x) for x,y in c]
fig, axarr = plt.subplots(2, sharex=True, figsize=(1400/120, 700/120))
palette = sns.color_palette('hls', n_colors=12)
clrs = [palette[d[0][1]-1] for d in c]
bp1 = sns.barplot(datesort, [y for x,y in c], palette = clrs, ax=axarr[0], linewidth=0)
axarr[0].set_xticklabels(names)

startdates = [x+y for x,y in zip(df2.date_time, [relativedelta(days=x) for x in df2.srch_booking_window])]
staydates = zip(startdates, df2.srch_length_of_stay)
newdates = []
for staydate in staydates:
    for day in range(staydate[1]):
        newdates.append(staydate[0] + relativedelta(days=day))
newdates = [(a.year, a.month, a.day) for a in newdates if a.year < 2013 or (a.year == 2013 and a.month < 11)]
newc = sorted(Counter(newdates).items(), key=lambda kv: kv[0][0]*10000 + kv[0][1]*100 + kv[0][2])
newdatesort = [kv[0][0]*10000 + kv[0][1]*100 + kv[0][2] for kv in newc]
newnames = [name(x) for x,y in newc]
newclrs = [palette[d[0][1]-1] for d in newc]
bp2 = sns.barplot(newdatesort, [y for x,y in newc], palette = newclrs, ax=axarr[1], linewidth=0)
axarr[1].set_xticklabels(newnames)

bp1.set_ylabel("Number of searches")
bp2.set_ylabel("Number of active bookings")

plt.savefig("dateplot", dpi=1000)


In [ ]:
df_train['srch_booking_window'].value_counts()

In [ ]:
sorted_book_window = dict(df_train.srch_id.value_counts().sort_values())
fig, ax = plt.subplots(figsize=(1200/120, 500/120))
perc =  [i for i in sorted_book_window.values()]
bp = sns.barplot(x=list(sorted_book_window.keys()),y=perc, palette='Spectral')
bp.set_ylabel("Num of booking")

for item in bp.get_xticklabels():
    item.set_rotation(90)
plt.subplots_adjust(bottom=0.5)

In [ ]:
site_id_sorted = dict(df_train.site_id.value_counts().sort_values())
fig, ax = plt.subplots(figsize=(1200/120, 500/120))
perc =  [i for i in site_id_sorted.values()]
bp = sns.barplot(x=list(site_id_sorted.keys()),y=perc, palette='Spectral')
bp.set_ylabel("Num of searches")
bp.set_xlabel("Site ID")
bp.set_yscale("log")
plt.subplots_adjust(bottom=0.5)

In [ ]:
perc

In [ ]:
df_train.site_id.value_counts()

In [ ]:
nrows = df_train.shape[0]
fig, ax = plt.subplots(figsize=(1200/120, 500/120))
d = df_train.isnull().sum().to_dict()
sorted_d = dict(sorted(d.items(), key=lambda kv: (kv[1],kv[0])))


perc =  [float(i*100/nrows) for i in sorted_d.values()]
bp = sns.barplot(x=list(sorted_d.keys()),y=perc, palette='Spectral')
bp.set_ylabel("Percentage missing")

for item in bp.get_xticklabels():
    item.set_rotation(90)

plt.subplots_adjust(bottom=0.5)
plt.savefig("barplot_missingvalues", dpi=400)